129L Section 3 <br>
Task 1: Turing machine for binary multiplication

In [1]:
class Tape(object):
    
    blank_symbol = " "
    
    def __init__(self,
                 tape_string = ""):
        self.__tape = dict((enumerate(tape_string)))
        # last line is equivalent to the following three lines:
        #self.__tape = {}
        #for i in range(len(tape_string)):
        #    self.__tape[i] = input[i]
        
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return Tape.blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 

        
class TuringMachine(object):
    
    def __init__(self, 
                 tape = "", 
                 blank_symbol = " ",
                 initial_state = "",
                 final_states = None,
                 transition_function = None):
        self.__tape = Tape(tape)
        self.__head_position = 0
        self.__blank_symbol = blank_symbol
        self.__current_state = initial_state
        if transition_function == None:
            self.__transition_function = {}
        else:
            self.__transition_function = transition_function
        if final_states == None:
            self.__final_states = set()
        else:
            self.__final_states = set(final_states)
        
    def get_tape(self): 
        return str(self.__tape)
    
    def step(self):
        char_under_head = self.__tape[self.__head_position]
        x = (self.__current_state, char_under_head)
        if x in self.__transition_function:
            y = self.__transition_function[x]
            self.__tape[self.__head_position] = y[1]
            if y[2] == "R":
                self.__head_position += 1
            elif y[2] == "L":
                self.__head_position -= 1
            self.__current_state = y[0]

    def final(self):
        if self.__current_state in self.__final_states:
            return True
        else:
            return False

In [5]:
class TuringMachine:
    def __init__(self, tape, states, initial_state, blank_symbol=' '):
        self.tape = list(tape)  # Tape as a list of symbols
        self.head = 0  # Head starts at the first position
        self.states = states  # Transition rules
        self.state = initial_state  # Initial state
        self.blank_symbol = blank_symbol  # Blank symbol
        self.tape_log = []  # Log of tape states

    def step(self):
        """Performs one step of the Turing machine."""
        symbol = self.tape[self.head]
        if (self.state, symbol) in self.states:
            # Get the transition rule for the current state and symbol
            next_state, write_symbol, move = self.states[(self.state, symbol)]
            self.tape[self.head] = write_symbol  # Write the symbol on the tape
            self.head += 1 if move == 'R' else -1 if move == 'L' else 0  # Move the head
            self.state = next_state  # Transition to the next state
            # Expand tape if the head moves beyond the current limits
            if self.head < 0:
                self.tape.insert(0, self.blank_symbol)
                self.head = 0
            elif self.head >= len(self.tape):
                self.tape.append(self.blank_symbol)
        else:
            # Halt if no valid transition
            self.state = 'HALT'
        # Log the current tape state
        self.tape_log.append(''.join(self.tape))

    def run(self, max_steps=1000):
        """Runs the Turing machine."""
        steps = 0
        while self.state != 'HALT' and steps < max_steps:
            self.step()
            steps += 1
        return ''.join(self.tape).strip(self.blank_symbol)

    def save_tape_log(self, filename):
        """Saves the tape log to a .dat file."""
        with open(filename, 'w') as f:
            for line in self.tape_log:
                f.write(line + '\n')

In [6]:
# Define the states and transitions for binary multiplication
# Example: multiply two numbers separated by '#' and store result on tape
states = {
    # Example transitions (must be expanded for full functionality)
    ('q0', '0'): ('q0', '0', 'R'),
    ('q0', '1'): ('q0', '1', 'R'),
    ('q0', '#'): ('q1', '#', 'R'),
    ('q1', '0'): ('q1', '0', 'R'),
    ('q1', '1'): ('q1', '1', 'R'),
    ('q1', ' '): ('HALT', ' ', 'N'),  # Example halting condition
}

In [7]:
# Initialize the Turing machine tape with binary numbers to multiply
binary1 = "110"  # Example: 6
binary2 = "101"  # Example: 5
tape = f"{binary1}#{binary2} "  # Input format
tm = TuringMachine(tape, states, initial_state='q0')

In [8]:
# Run the Turing machine
tm.run()

# Save the tape log to a .dat file
filename = f"{binary1}_times_{binary2}.dat"
tm.save_tape_log(filename)

print(f"Result on tape: {''.join(tm.tape).strip(tm.blank_symbol)}")
print(f"Tape log saved to {filename}")

Result on tape: 110#101
Tape log saved to 110_times_101.dat


In [9]:
class TuringMachine:
    def __init__(self, tape, transitions, initial_state, halt_states):
        self.tape = list(tape)  # Convert tape to a mutable list
        self.head = 0           # Start at the leftmost position
        self.state = initial_state
        self.transitions = transitions
        self.halt_states = halt_states
        self.tape_history = []  # To store tape snapshots

    def step(self):
        """Perform a single step of the Turing machine."""
        symbol = self.tape[self.head]
        key = (self.state, symbol)

        if key in self.transitions:
            new_state, write_symbol, direction = self.transitions[key]
            self.tape[self.head] = write_symbol
            self.state = new_state
            self.head += 1 if direction == 'R' else -1

            # Expand tape if head moves beyond bounds
            if self.head < 0:
                self.tape.insert(0, 'B')  # Add blank to the left
                self.head = 0
            elif self.head >= len(self.tape):
                self.tape.append('B')  # Add blank to the right

        else:
            self.state = 'HALT'  # Transition to halt if no valid transition

        # Record the current state of the tape
        self.tape_history.append((self.state, ''.join(self.tape), self.head))

    def run(self):
        """Run the Turing machine until it halts."""
        while self.state not in self.halt_states:
            self.step()

    def save_history(self, filename):
        """Save the tape history to a .dat file."""
        with open(filename, 'w') as f:
            for state, tape, head in self.tape_history:
                # Format: State, tape, head position
                f.write(f"{state}\t{tape}\t{head}\n")

# Binary multiplication Turing machine configuration
transitions = {
    # Initial state: Mark the first digit in the first number
    ('q0', '1'): ('q1', 'X', 'R'),
    ('q0', '0'): ('q1', 'Y', 'R'),

    # Move to the separator '#'
    ('q1', '1'): ('q1', '1', 'R'),
    ('q1', '0'): ('q1', '0', 'R'),
    ('q1', '#'): ('q2', '#', 'R'),

    # Mark the first digit in the second number
    ('q2', '1'): ('q3', 'X', 'L'),
    ('q2', '0'): ('q3', 'X', 'L'),

    # Move back to the separator '#'
    ('q3', '1'): ('q3', '1', 'L'),
    ('q3', '0'): ('q3', '0', 'L'),
    ('q3', '#'): ('q4', '#', 'L'),

    # Move back to the first number
    ('q4', 'X'): ('q4', 'X', 'L'),
    ('q4', '1'): ('q5', 'X', 'R'),
    ('q4', '0'): ('q5', 'X', 'R'),

    # Repeat the process or finish when complete
    ('q5', '1'): ('q5', '1', 'R'),
    ('q5', '0'): ('q5', '0', 'R'),
    ('q5', '#'): ('q6', '#', 'R'),

    # Handle final states and halting logic here...
}

# Initial configuration
tape = "101#110$"
initial_state = 'q0'
halt_states = ['HALT']

# Instantiate the Turing machine
tm = TuringMachine(tape, transitions, initial_state, halt_states)

# Run the Turing machine
tm.run()

# Save the history to a .dat file
filename = "101x110.dat"  # Replace with dynamic input as needed
tm.save_history(filename)

print(f"Multiplication complete. Tape history saved to {filename}.")

Multiplication complete. Tape history saved to 101x110.dat.
